# Predict Turbofan Degradation

In this tutorial, build a machine learning application to predict turbofan engine degradation. This application is structured into three important steps:

- Prediction Engineering
- Feature Engineering
- Machine Learning

In the first step, create new labels from the data by using [Compose](https://compose.alteryx.com/). In the second step, generate features for the labels by using [Featuretools](https://featuretools.alteryx.com/). In the third step, search for the best machine learning pipeline using [EvalML](https://evalml.alteryx.com/). After working through these steps, you should understand how to build machine learning applications for real-world problems like forecasting demand.

**Note: In order to run this example, you should have Featuretools 1.4.0 or newer and EvalML 0.41.0 or newer installed.**

In [ ]:
from demo.turbofan_degredation import load_sample
from matplotlib.pyplot import subplots
import composeml as cp
import featuretools as ft
import evalml

Use a dataset provided by [NASA](https://www.kaggle.com/behrad3d/nasa-cmaps) simulating turbofan engine degradation. In the dataset, there is data about engines that have been monitored over time. Each engine had operational settings and sensor measurements recorded over a number of cycles. The remaining useful life (RUL) is the amount of cycles an engine has left before it needs maintenance. What makes this dataset special is that the engines run all the way until failure, giving us precise RUL information for every engine at every point in time. The model you build in this tutorial predicts RUL.

In [ ]:
df = load_sample()
df.head()

## Prediction Engineering

> Which range is the RUL of a turbofan engine in?

In this prediction problem, you want to group the RUL data into ranges, then predict which range the RUL is in. You can make variations in the ranges to create different prediction problems. For example, the ranges could be manually defined (0 - 150, 150 - 300, etc.) or based on the quartiles from historical observations. Bin the RUL to make variations, helping you explore different scenarios that are crucial for making better decisions.

### Defining the Labeling Function

Let’s start by defining the labeling function of an engine that calculates the RUL. Given that engines run all the way until failure, the RUL is just the remaining number of observations. Our labeling function will be used by a label maker to extract the training examples.

In [ ]:
def rul(ds):
    return len(ds) - 1

### Representing the Prediction Problem

Represent the prediction problem by creating a label maker with the following parameters:

- The `target_dataframe_index` as the column for the engine ID, since you want to process records for each engine.
- The `labeling_function` as the function you defined previously.
- The `time_index` as the column for the event time.

In [ ]:
lm = cp.LabelMaker(
    target_dataframe_index="engine_no",
    labeling_function=rul,
    time_index="time",
)

### Finding the Training Examples

Run a search to get the training examples by using the following parameters:

- The records sorted by the event time, since the search expects the records to be sorted chronologically. Otherwise, an error occurs.
- `num_examples_per_instance` as the number of training examples to find for each engine.
- `minimum_data` as the amount of data to use to make features for the first training example.
- `gap` as the number of rows to skip between examples. This is done to cover different points in time of an engine.

You can easily tweak these parameters and run more searches for training examples as the requirements of our model change.

In [ ]:
lt = lm.search(
    df.sort_values("time"),
    num_examples_per_instance=20,
    minimum_data=5,
    gap=20,
    verbose=False,
)

lt.head()

The output from the search is a label times table with three columns:

- The engine ID associated to the records. There can be many training examples generated from each engine.
- The event time of the engine. This is also known as a cutoff time for building features. Only data that existed beforehand is valid to use for predictions.
- The value of the RUL. This is calculated by the labeling function.

At this point, you only have continuous values of the RUL. As a helpful reference, you can print out the search settings that were used to generate these labels.

In [ ]:
lt.describe()

You can also get a better look at the values by plotting the distribution and the cumulative count across time.

In [ ]:
%matplotlib inline
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

With the continuous values, you can explore different ranges without running the search again. In this case, use quartiles to bin the values into ranges.

In [ ]:
lt = lt.bin(4, quantiles=True, precision=0)

When you print out the settings again, you can now see that the description of the labels has been updated and reflects the latest changes.

In [ ]:
lt.describe()

Look at the new label distribution and cumulative count across time.

In [ ]:
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

## Feature Engineering

In the previous step, you generated the labels. The next step is to generate features.

### Representing the Data

Let’s start by representing the data with an EntitySet. That way, you can generate features based on the relational structure of the dataset. You currently have a single table of records where one engine can have many records. This one-to-many relationship can be represented by normalizing an engine dataframe. The same can be done for other one-to-many relationships. Because you want to make predictions based on the engine, you should use this engine dataframe as the target for generating features.

In [ ]:
es = ft.EntitySet("observations")

es.add_dataframe(
    dataframe=df.reset_index(),
    dataframe_name="records",
    index="id",
    time_index="time",
)

es.normalize_dataframe(
    base_dataframe_name="records",
    new_dataframe_name="engines",
    index="engine_no",
)

es.normalize_dataframe(
    base_dataframe_name="records",
    new_dataframe_name="cycles",
    index="time_in_cycles",
)

es.plot()

### Calculating the Features

Now you can generate features by using a method called Deep Feature Synthesis (DFS). That method automatically builds features by stacking and applying mathematical operations called primitives across relationships in an entityset. The more structured an entityset is, the better DFS can leverage the relationships to generate better features. Run DFS with these parameters:

- `entityset` as the entityset we structured previously.
- `target_dataframe_name` as the engine dataframe.
- `cutoff_time` as the label times that we generated previously. The label values are appended to the feature matrix.

In [ ]:
fm, fd = ft.dfs(
    entityset=es,
    target_dataframe_name="engines",
    agg_primitives=["sum"],
    trans_primitives=[],
    cutoff_time=lt,
    cutoff_time_in_index=True,
    include_cutoff_time=False,
    verbose=False,
)

fm.head()

There are two outputs from DFS: a feature matrix and feature definitions. The feature matrix is a table that contains the feature values with the corresponding labels based on the cutoff times. Feature definitions are features in a list that can be stored and reused later to calculate the same set of features on future data.

## Machine Learning

In the previous steps, generate the labels and features. The final step is to build the machine learning pipeline.

### Splitting the Data

Start by extracting the labels from the feature matrix and splitting the data into a training set and a holdout set.

In [ ]:
fm.reset_index(drop=True, inplace=True)
y = fm.ww.pop("rul").cat.codes

splits = evalml.preprocessing.split_data(
    X=fm,
    y=y,
    test_size=0.2,
    random_seed=2,
    problem_type="multiclass",
)

X_train, X_holdout, y_train, y_holdout = splits

### Finding the Best Model

Run a search on the training set to find the best machine learning model. During the search process, predictions from several different pipelines are evaluated to find the best pipeline.

In [ ]:
automl = evalml.AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    problem_type="multiclass",
    objective="f1 macro",
    random_seed=0,
    allowed_model_families=["catboost", "random_forest"],
    max_iterations=3,
)

automl.search()

Once the search is complete, you can print out information about the best pipeline found, like the parameters in each component.

In [ ]:
automl.best_pipeline.describe()
automl.best_pipeline.graph()

Score the model performance by evaluating predictions on the holdout set.

In [ ]:
best_pipeline = automl.best_pipeline.fit(X_train, y_train)

score = best_pipeline.score(
    X=X_holdout,
    y=y_holdout,
    objectives=["f1 macro"],
)

dict(score)

From the pipeline, you can see which features are most important for predictions.

In [ ]:
feature_importance = best_pipeline.feature_importance
feature_importance = feature_importance.set_index("feature")["importance"]
top_k = feature_importance.abs().sort_values().tail(20).index
feature_importance[top_k].plot.barh(figsize=(8, 8), fontsize=14, width=0.7);

### Making Predictions

You are ready to make predictions with our trained model. Start by calculating the same set of features by using the feature definitions. Use a cutoff time based on the latest information available in the dataset.

In [ ]:
fm = ft.calculate_feature_matrix(
    features=fd,
    entityset=es,
    cutoff_time=ft.pd.Timestamp("2001-01-08"),
    cutoff_time_in_index=True,
    verbose=False,
)

fm.head()

Now predict which one of the four ranges the RUL is in.

In [ ]:
y_pred = best_pipeline.predict(fm)
y_pred = y_pred.values

prediction = fm[[]]
prediction["rul (estimate)"] = y_pred
prediction.head()

### Next Steps

You have completed this tutorial. You can revisit each step to explore and fine-tune the model using different parameters until it is ready for production. For more information about how to work with the features produced by Featuretools, take a look at [the Featuretools documentation](https://featuretools.alteryx.com/). For more information about how to work with the models produced by EvalML, take a look at [the EvalML documentation](https://evalml.alteryx.com/).